# Models
(arranjar um nome melhor para o notebook)

Notebook para o treinamento de modelos

## Data Preparation

Apenas realizar modificações necessárias para os modelos como *splitting* dos datasets e normalização. Toda a limpeza de dados deve ser feita em **/src/data-engineering/data-preparation**

### Train-test-validation split

### Data Normalization

## Training

## Model Evaluation